# Running PyNN Simulations on SpiNNaker

## Spiking Neural Networks
Biological neurons have been observed to produce sudden and short increases in voltage, commonly
referred to as spikes. The spike causes a charge to be transferred across the synapse between neurons.
The charge from all the presynaptic neurons connected to a postsynaptic neuron builds up, until that
neuron releases the charge itself in the form of a spike. The spike travels down the axon of the neuron
which then arrives after some delay at the synapses of that neuron, causing charge to be passed forward to
the next neuron, where the process repeats.

<img src="SpikingNeuralNetworks.png" />

The charge can result in either an excitatory response, in which the membrane voltage of the postsynaptic
neuron increases or an inhibitory response, in which the membrane voltage of the postsynaptic neuron
decreases as a result of the spike.

## The PyNN Neural Network Description Language
PyNN is a language for building spiking neural network models. PyNN models can then be run on a
number of simulators without modification (or with only minor modifications), including SpiNNaker. The
basic steps of building a PyNN network are as follows:

1. Setup the simulator
1. Create the neural populations
1. Create the projections between the populations
1. Setup data recording
1. Run the simulation
1. Retrieve and process the recorded data

An example of this is shown below.

In [ ]:
import spynnaker.pyNN as sim
import pyNN.utility.plotting as plot
import matplotlib.pyplot as plt

# Setup the Simulator
sim.setup(timestep=1.0)
sim.set_number_of_neurons_per_core(sim.IF_curr_exp, 100)

# Create the neural populations
pop_1 = sim.Population(1, sim.IF_curr_exp(), label="pop_1")
input = sim.Population(1, sim.SpikeSourceArray(spike_times=[0]), label="input")

# Create projections between the populations
input_proj = sim.Projection(input, pop_1, sim.OneToOneConnector(),
synapse_type=sim.StaticSynapse(weight=5, delay=1))

# Setup data recording
pop_1.record(["spikes", "v"])

# Run the simulation
simtime = 10
sim.run(simtime)

# Retrieve and process the recorded data
neo = pop_1.get_data(variables=["spikes", "v"])
spikes = neo.segments[0].spiketrains
print(spikes)
v = neo.segments[0].filter(name='v')[0]
print(v)
sim.end()

plot.Figure(
    # plot voltage for first ([0]) neuron
    plot.Panel(v, ylabel="Membrane potential (mV)",
               data_labels=[pop_1.label], yticks=True, xlim=(0, simtime)),
    # plot spikes (or in this case spike)
    plot.Panel(spikes, yticks=True, markersize=5, xlim=(0, simtime)),
    title="Simple Example",
    annotations="Simulated with {}".format(sim.name())
)

This example runs using a 1.0ms timestep. It creates a single input source (A SpikeSourceArray ) sending
a single spike at time 0, connected to a single neuron (with model IF_curr_exp ). The connection is
weighted, so that a spike in the presynaptic neuron sends a fixed (or Static) current of 5 nanoamps (nA) to
the excitatory synapse of the postsynaptic neuron, with a delay of 1 millisecond. The spikes and the
membrane voltage are recorded, and the simulation is then run for 10 milliseconds. Graphs are then
created of the membrane voltage and the spikes produced.

### Populations and Neuron Models
In PyNN, the neurons are declared in terms of a population of a number of neurons with similar properties.
PyNN provides a number of standard neuron models. One of the most basic of these is known as the
Leaky Integrate and Fire (LIF) model, and this is used above ( IF_curr_exp ). This models the neuron as a
resistor and capacitor in parallel; as charge is received, this builds up in the capacitor, but then leaks out
through the resistor. In addition, a threshold voltage is defined; if the voltage reaches this value, a spike is
produced. For a time after this, known as the refractory period , the neuron is not allowed to spike again.
Once this period has passed, the neuron resumes operation as before. Additionally, the synapses are
modelled using an exponential decay of the received current input (5 nA in the above example); the weight
of the current is added over a number of timesteps, with the current decaying exponentially between each.
A longer decay rate will result in more charge being added overall per spike that crosses the synapse.

In the above example, the default parameters of the IF_curr_exp are used. These are shown below.

```
lif_curr_exp_params = {
    'cm': 1.0,          # The capacitance of the LIF neuron in nano-Farads
    'tau_m': 20.0,      # The time-constant of the RC circuit, in milliseconds
    'tau_refrac': 0.1,  # The refractory period, in milliseconds
    'v_reset': -65.0,   # The voltage to set the neuron at immediately after a spike
    'v_rest': -65.0,    # The ambient rest voltage of the neuron
    'v_thresh': -50.0,  # The threshold voltage at which the neuron will spike
    'tau_syn_E': 5.0,   # The excitatory input current decay time-constant
    'tau_syn_I': 5.0,   # The inhibitory input current decay time-constant
    'i_offset': 0.0,    # A base input current to add each timestep
}
```

PyNN supports both current-based models and conductance-based models. In conductance models, the
input is measured in microSiemens, and the effect on the membrane voltage also varies with the current
value of the membrane voltage; the higher the membrane voltage, the more input is required to cause a
spike. This is modelled as the reversal potential of the synapse; when the membrane potential equals the
reversal potential, no current will flow across the synapse. A conductance-based version of the LIF model
is provided, which, in addition to the above parameters, also supports the following.
```
lif_cond_exp_extra_params = {
    'e_rev_E': 0.,    # The reversal potential of the excitatory synapse
    'e_rev_I': -80.0  # The reversal potential of the inhibitory synapse
}
```

The initial value of the state variables of the neural model can also be set (such as the membrane voltage).
This is done via the initialize function of the population, which takes the name of the state variable (e.g. v
for the membrane voltage), and the value to be assigned e.g. the initial voltage can be set to -65.0mV as follows.
```
pop_1.initialize(v=-65.0)
```

In addition to neuron models, the PyNN language also supports some utility models, which can be used to
simulate inputs into the network with defined characteristics. These include:
 - SpikeSourceArray - this sends spikes at predetermined intervals defined by spike_times . In general, PyNN forces each of the neurons in the population to spike at the same time, and so spike_times is an array of times, but sPyNNaker also allows spike_times to be an array of arrays, each defining the the times at which each neuron should spike e.g. spike_times =[[0], [1]] means that the first neuron will spike at 0ms and the second at 1ms.
 - SpikeSourcePoisson - this sends spikes at random times with a mean rate of rate spikes per second, starting at start_time (0.0ms by default) for a duration of duration milliseconds (the whole simulation by default).

### Population Structure
Populations can have additional structure, which means that the way in which the neurons are laid out in space is specified.  This information can then be used by connectors to e.g. determine the distance between neurons and therefore make different connections based on that information.  Additionally the SpiNNaker software can make use of this information to efficiently reduce the connections between cores on the machine and therefore improve the performance of the simulation.

The structure is specified using ```Population(..., structure=...)``` when definiting the Population.  There are a few options, but some common ones include:
 - ```Grid2D(ratio_x_y)``` - this specifies a 2D layout of neurons, where ```ratio_x_y``` is the ratio of the x-coordinates to the y-coordinates, most easily specified as ```width / height``` e.g. ```p.Population(640 * 480, p.IF_curr_exp(), structure=p.Grid2D(640 / 480)```.
 - ```Grid3D(ratio_x_y, ratio_x_z)``` - this specifies a 3D layour of neurons, where there are now ratios between x and y coordinates and x and z coordinates.

### Projections and Connectors
Populations of neurons are joined together using a Projection . This is a directed connection where spikes
are sent from the source, or pre-population and the target, or post-population. The projection between
populations of neurons has a connector , which describes the connectivity between the individual neurons in
the populations. Some common connectors include:

 - OneToOneConnector - each presynaptic neuron connects to one postsynaptic neuron (there should be the same number of neurons in each population).
 - AllToAllConnector - all presynaptic neurons connect to all postsynaptic neurons.
 - FixedProbabilityConnector - each presynaptic neuron connects to each postsynaptic neuron with a given fixed probability p_connect.
 - FromListConnector - the exact connectivity is described by conn_list , which is a list of tuples (pre_synaptic_neuron_id, post_synaptic_neuron_id, weight, delay) or just (pre_synaptic_neuron_id, post_synaptic_neuron_id). Note: All tuples must be the same length. If weight, delay are included the ones supplied through the synapse_type parameter of the Projection are ignored.
 - FixedTotalNumberConnector - an exact number of connections n_synapses are made, drawn at random from the possible connections, with replacement. Note that this means that connections can be repeated.

As well as a connector the Projection must also have a synapse_type which determines how the synapse
behaves when spikes are received. For example a StaticSynapse which has fixed weights and delays is
specified as shown below.
synapse_type=sim.StaticSynapse(weight=0.75, delay=1.0)

### Random Parameters
Commonly, random weights and/or delays are used. To specify this, the value of the weight or delay of the
synapse type are set to a RandomDistribution ; note that the FromListConnector should then be specified
with tuples of only ( pre_synaptic_neuron_id, post_synaptic_neuron_id ). The RandomDistribution supports
several parameters via the parameters argument, depending on the value of the distribution argument
which identifies the distribution type. The supported distributions include a ‘uniform’ distribution, with
parameters of low (the minimum value) and high (the maximum value); and a ‘normal’ distribution with
parameters of mu (the mean) and sigma (the standard deviation); as well as a ‘normal_clipped’ distribution,
which takes the same parameters as ‘normal’ but with the addition of boundary parameters of low and high - this is often useful for keeping the delays within range allowed by the simulator. The RandomDistribution
can also be used when specifying neural parameters, or when initialising state variables.

### Recording Data
All the Populations in a simulation can be recorded; the data which can be recorded is dependent on the
simulation model. In general, all of the neuron models in PyNN allow the recording of the times at which
each neuron spikes, spikes , and the membrane potential, v . In contrast, the input models (i.e.
SpikeSourceArray and SpikeSourcePoisson) only allow the recording of spikes. On SpiNNaker, our neuron
models additionally allow the recording of the neuron input using gsyn ; technically, PyNN reserves this for
the recording of synaptic conductance in models which support this (e.g. IF_cond_exp) but we also allow
the recording of the synaptic currents in models such as IF_curr_exp.

### Running the Simulation
Once the network has been described and the data to be recorded has been selected, the simulation can
be started by calling run with the duration that the simulation is to be executed for. The run method can be
called multiple times in sequence to run for further durations. In between each run, it is possible to change
parameters of the network; at present SpiNNaker simulations only support the changing of the parameters
of the populations, such as changing the i_offset to adjust the input to the neurons. It is also possible to
retrieve recorded data (see below) in between runs.
If you want to reset the simulation back to time 0 this can also be done using the reset call. At this point, it
is now possible to make further changes in SpiNNaker simulations, such as adding Populations and
Projections; note that these changes will result in a full remapping which takes longer than changes to the
parameters.

### Retrieving and Plotting Data
Once the simulation has been run, the Population get_data method can be used to retrieve the recorded
data in the form of a Neo object (see https://neuralensemble.org/neo/ ). Each Neo object has a list of
segments, one per reset-run cycle (so there will only be one if you never call reset). The content of each of
the segments depends on the data recorded and requested.
Spike data is accessible via the .spiketrains property; there is one SpikeTrain for each neuron in the
population. Each SpikeTrain can be treated as a numpy array of the times during the simulation at which
the neuron spiked.

Other data is accessible via the .filter(name=<signal_name>) method, where <signal_name> is the name of
the data item to retrieve (i.e. v for the membrane voltage). This returns an array of AnalogSignalArray
objects; in the case of SpiNNaker there will only be one element in this array as all data is gathered
together into a single array, thus the 0th element can always be used (e.g. .filter(name=’v’)[0]) . The
AnalogSignalArray in turn contains a list of AnalogSignalArray objects, one for each neuron. Each of these
sub-arrays contains the list of values of the signal, one per time-step. Both SpikeTrain and
AnalogSingnalArray objects extend Quanties arrays; this means that they come with the unit of the values
as well. The SpikeTrain values are all in milliseconds, and the membrane voltages are in millivolts. These
objects also hold additional metadata.

The results of Neo.segements[0].spiketrains and Neo.segements[0].filter(name=)[0] can be passed to the
pyNN.utility.plotting.Panel as shown in the example above. The module spynnaker8.spynakker_plotting
contains a SpynakkerPanel object can also be used in the same way for slightly faster spike plots and to
display heatmap-style plots for analog signal data.

## Spike-Time-Dependent Plasticity
STDP plasticity is a form of learning that depends upon the timing between the spikes of two neurons
connected by a synapse. It is believed to be the basis of learning and information storage in the human
brain.

In the case where a presynaptic spike is followed closely by a postsynaptic spike, then it is presumed that
the presynaptic neuron caused the spike in the postsynaptic neuron, and so the weight of the synapse
between the neurons is increased. This is known as potentiation.

If a postsynaptic spike is emitted shortly before a presynaptic spike is emitted, then the presynaptic spike
cannot have caused the postsynaptic spike, and so the weight of the synapse between the neurons is
reduced. This is known as depression.

The size of the weight change depends on the relative timing of the presynaptic and postsynaptic spikes; in
general, the change in weight drops off exponentially as the time between the spikes gets larger, as shown
in the following figure [Sjöström and Gerstner (2010), Scholarpedia]. However, different experiments have
highlighted different behaviours depending on the conditions (e.g. [Graupner and Brunel (2012), PNAS]).
Other authors have also suggested a correlation between triplets and quadruplets of presynaptic and
postsynaptic spikes to trigger synaptic potentiation or depression.

<img src="STDP.png"/>


### STDP in PyNN
The steps for creating a network using STDP are much the same as previously described, with the main
difference being that some of the projections use a STDPMechanism to describe the plasticity. Below is an
example of the creation of a projection with STDP.

```
timing_rule = sim.SpikePairRule(tau_plus=20.0, tau_minus=20.0, A_plus=0.5, A_minus=0.5)
weight_rule = sim.AdditiveWeightDependence(w_max=5.0, w_min=0.0)
stdp_model = sim.STDPMechanism(timing_dependence=timing_rule, weight_dependence=weight_rule, weight=0.0, delay=5.0)
stdp_projection = sim.Projection(input, pop_1, sim.OneToOneConnector(), synapse_type=stdp_model)
```

In this example, firstly the timing rule is created. In this case, it is a SpikePairRule , which means that the
relative timing of the spikes that will be used to update the weights will be based on pairs of presynaptic
and postsynaptic spikes. This rule has four parameters. The parameters tau_plus and tau_minus describe
the respective exponential decay of the size of the weight update with the time between presynaptic and
postsynaptic spikes. Note that the decay can be different for potentiation (defined by tau_plus ) and
depression (defined by tau_minus ). The parameters A_plus and A_minus which define the maximum
weight to respectively add during potentiation or subtract during depression.

The next thing defined is the weight update rule. In this case it is a AdditiveWeightDependence , which
means that the weight will be updated by simply adding to the current weight. This rule requires the
parameters w_max and w_min , which define the maximum and minimum weight of the synapse
respectively, Note that the actual amount added or subtracted will depend additionally on the timing of the
spikes, as determined by the timing rule.

In addition, there is also a MultiplicativeWeightDependence supported, which means that the weight
change depends on the difference between the current weight and w_max for potentiation, and w_min for
depression. The value of A_plus and A_minus are then respectively multiplied by this difference to give the
maximum weight change; again the actual value depends on the timing rule and the time between the
spikes.

The timing and weight rules are combined with weight and delay into a single STDPMechanism object
which describes the overall desired mechanism. Note that the projection still requires the specification of a
connector. This connector is still used to describe the overall connectivity between the neurons of the preand
post-populations. It is preferable that the initial weights fall between w_min and w_max ; it is not an
error if they do not, but when the first update is performed, the weight will be changed to fall within this
range.

Note that on SpiNNaker, although multiple projections to the same target population can be specified with
STDP, the restrictions on the current software are that all those projections must use the same rules with
the exact same parameters. This is due to the restrictions of the local memory available on each core,
reducing the amount of data that can be held for the parameters.
Note: In the implementation of STDP on SpiNNaker, the plasticity mechanism is only activated when the
second presynaptic spike is received at the postsynaptic neuron. Thus at least two presynaptic spikes are
required for the mechanism to be activated.

### Getting Synaptic Data
The weights and delays assigned to a projection can be retrieved using the Projection’s get method,
specifying the data items to get, including ‘weight’, ‘delay’ and the parameters of the STDP Mechanism, and
the format they are retrieved using. The data formats supported are ‘list’ format, where the return value
consists of a list of tuples of the selected values; and ‘array’ where each value is returned in a
two-dimensional matrix indexed by the source neurons in the pre-population, and the target neurons in the
post-populations. In the ‘list’ result, each tuple additionally contains the source and target neuron ids as the
0th and 1st values in the tuple. In the ‘array’ result, missing connections are represented as ‘NaN’ (not a
number) and positions where there are multiple connections have their values summed.

Note that on SpiNNaker, it is possible to retrieve the projection data before calling the PyNN run function,
but that this data cannot be examined until after run has been called. This is because the individual
connectivity data is not generated until the run function is called.

## Non-PyNN Features of SpiNNaker Software
These features are somewhat related to the PyNN language and use the PyNN format to describe them, but are actually SpiNNaker specific and so will not work with other PyNN implementations.  This has been done because these features do not exist in other simulators directly.

### Set Number of Neurons Per Core
It is possible to set the number of neurons on each core for a given neuron model, or even for an individual Populations.  This can be either 1D or 2D (or more), though note that when done with a model, this much match all Populations; thus if there is a mixture of dimensions, it may be easier to set this on each Population separately.  Additionally, when anything other than a 1D value is set, the size of the Population in each dimension must be divisible by the respective number of neurons per core in that dimension.  An example of the global model-focussed command setting this for the LIF model is as follows:
```
# Set the number of neurons to 128 per core
sim.set_number_of_neurons_per_core(sim.IF_curr_exp, 128)

# Set the number of neurons to 10x10 per core
sim.set_number_of_neurons_per_core(sim.IF_curr_exp, (10, 10))
```

When doing this when creating a Population, the syntax is as follows:
```
pop = sim.Population(100, sim.IF_curr_exp(), structure=sim.Grid2D(10 / 10))
pop.set_max_atoms_per_core((5, 5))
```

### Convolution Connector
The Convolution Connector is a specific way of providing a convolutional kernel as connector in a Projection between Populations.  Like other Projection connectors, this describes the way in which the weights are applied to the target neurons based on which source neurons spike.  The main difference is that this has been made to be efficiently executed within a SpiNNaker simulation; in more detail, rather than explicitly expanding the connections between the neurons, the kernel itself is stored in the local memory of the target core, and then applied through processing when spikes are received.  This has the additional effect that the number of neurons per core can be increased somewhat; with a convolution connector, the target can have around 1024 neurons per core.

An example of a network with a convolution connector is shown below.  This network simply sends a single spike from the centre pixel of the input which is then fed through the kernel to affect the output pixel neuron voltages; none of the output pixels are expected to spike in this particular case.

In [ ]:
import numpy
sim.setup(1.0)

# Make a kernel and convolution connector
k_shape = numpy.array([5, 5], dtype='int32')
k_size = numpy.prod(k_shape)
kernel = (numpy.arange(k_size) - (k_size / 2)).reshape(k_shape) * 0.1
conn = sim.ConvolutionConnector(kernel)

# Start with an input shape, and deduce the output shape
in_shape = (11, 11)
out_shape = conn.get_post_shape(in_shape)
n_input = numpy.prod(in_shape)
n_output = numpy.prod(out_shape)

# Make a 2D source that spikes at the middle if the input shape
spike_idx = ((in_shape[1] // 2) * in_shape[0]) + (in_shape[1] // 2)
spike_times = [[1.0] if i == spike_idx else [] for i in range(n_input)]

# Note the structure=sim.Grid2D, which makes this a 2D population
src = sim.Population(
    n_input, sim.SpikeSourceArray, {'spike_times': spike_times},
    label='input spikes', structure=sim.Grid2D(in_shape[0] / in_shape[1]))
# Note we also need to do something like this to make it split in 2D
src.set_max_atoms_per_core((in_shape[0], inshapt[1]))

# Make a 2D target Population, make it split correctly and record it
output = sim.Population(
    n_output, sim.IF_curr_exp(), label="out",
   structure=sim.Grid2D(out_shape[0] / out_shape[1]))
output.set_max_atoms_per_core((out_shape[0], out_shape[1]))
output.record('v')

# Connect the two populations with the convolution.  Note the use
# of the synapse_type to ensure fast convolutional processing is
# done
sim.Projection(src, output, conn, sim.Convolution())

# Run the simulation
sim.run(10)
sim.end()

After setting up the simulator, this script first creates a convolution connector based on a given kernel.  In this case, the kernel is a 5x5 grid where the values increase from the top left corner to the bottom right corner.  This is then stored in the connector to be used in the simulation.

When a convolution connector is in use, the connector determines the allowed size of the target population based on the kernel size and the size of the source Population.  The connector allows this to be computed, but it will also be checked.  The next few lines use the connector to work out the output shape from the input, and also the number of neurons in the input and output populations.

Once this has been done, the position of the middle pixel of the input is determined and a spike array is make in which that neuron specifically spikes once.  

This is then used in a Population, and an output Population of neurons is then created.  Note that these Populations are set up using a 2D structure, which helps the software to efficiently route signals between the cores where the source and target end up.  In this example, each of the Populations will actually be on a single core so this is less crticial here, but with larger input and output Populations, this can reduce the number of spikes each core has to process.

The final step is to connect the Populations using a Projection.  This works in much the same way as other PyNN Projections, with the one difference being that instead of using a standard PyNN synapse type (such as `sim.StaticSynapse`), this uses a convolution-specific synapse type of `sim.Convolution()`.  This is required and an error will be generated if this is not done.

### Pool Dense Connector
The Pool Dense connector allows the specification of every weight between source and target, after applying a pool and a stride operation to the source.  Like the Convolution connector above, SpiNNaker performs the operation efficiently.  An example network is shown below.  This demonstrates which weights are used for which part of the pooled input to output mapping.

In [ ]:
sim.setup(1.0)

kernel = [[0.1, 0.2, 0.3, 0.4], 
          [0.1, 0.2, 0.3, 0.4], 
          [0.1, 0.2, 0.3, 0.4], 
          [0.1, 0.2, 0.3, 0.4]]
conn = sim.PoolDenseConnector(kernel, pool_shape=(2, 2), pool_stride=(2, 2))
ssa = sim.Population(16, sim.SpikeSourceArray([1.0]), structure=sim.Grid2D(4 / 4))
ssa.set_max_atoms_per_core((4, 4))
pop = sim.Population(4, sim.IF_curr_exp(v_reset=0.0, v_rest=0.0, v_thresh=100.0, tau_syn_E=1.0, cm=1.0, tau_m=1.0), 
                     structure=sim.Grid2D(2 / 2))
pop.initialize(v=0.0)
pop.set_max_atoms_per_core((2, 2))
pop.record("v")

sim.Projection(ssa, pop, conn, sim.PoolDense())

sim.run(10)
sim.end()

This first creates a kernel which will work for the mapping from a 4x4 input to a 2x2 output with a stride of 2x2.  This means that output neuron (0, 0) will get inputs from the first 2x2 input neurons only and so on over the population, with each output neuron getting input from a separate 2x2 input square.  The weights of those inputs are then the columns of the kernel e.g. the first column will be used for the mapping of the first 4 input neurons to the first output neuron.

This then proceeds much like the convolution network, except that the array is sending a spike to every input neuron every timestep.  Note that the correct number of weights must be specified in the kernel for the mapping.

### Structural Plasticity
The current implementation of Structural plasticity on SpiNNaker is specifically that it allows the rewiring of connections from a specific pre-population to a post-population, with a given maximum number of connections allowed to each post-neuron, ```s_max```.  On each rewiring step, the frequency of which is determined by ```f_rew```, a random post-neuron is chosen, and a random slot for that neuron from the ```s_max``` slots.  Once this is chosen the next step depends on whether there is already a connection in the slot or not:
- If there is no connection, a pre-neuron is chosen using the Partner Selection rule.  There are currently two such possible rules:
  - `sim.RandomSelection()` - this chooses any pre-neuron at random.
  - `sim.LastNeuronSelection()` - this chooses at random between neurons that spiked in the last timestep.
  
  Once partner is selected:
  - If the pre-neuron already has a connection to the post-neuron, and ```with_replacement``` is False, the connection is rejected.  
  - Otherwise, the Formation rule is consulted to determine whether a formation happens or not based on a probability that varies with the chosen pre and post neurons.  The only current rule is `sim.DistanceDependentFormation()`, which assumes that the pre and post populations are formed into a grid (note this doesn't currently tie in with the structure) and then uses the manhattan distance between the neurons when the pre is projected onto the post as a probability against whether the formation will happen, with closer neurons more likely to form a connection.
- If there is a connection, the Elimination rule is consulted to determine if the connection should be removed based again on a probability.  The only current rule for elimination is `sim.RandomByWeightElimination()`, where the probability is determined by the weight of the connection.  If this is above a threshold, one probability is used, and if below a different probability is used.

Structural plasticity can also be combined with STDP.  A simple example is shown below.  Note that this includes the ```download_synapses``` option.  This speeds up the reading of synapses by pre-reading them from the machine at the end of a simulation.  This is not necessary to read the synapses, but does speed up the process.

In [ ]:
sim.setup(1.0)
stim = sim.Population(1, sim.SpikeSourceArray(numpy.array(range(0, 10, 2))), label="stim")
pop = sim.Population(1, sim.IF_curr_exp(), label="pop")
proj = sim.Projection(
    stim, pop, sim.FromListConnector([]), sim.StructuralMechanismSTDP(
        partner_selection=sim.LastNeuronSelection(),
        formation=sim.DistanceDependentFormation([1, 1], 1.0),
        elimination=sim.RandomByWeightElimination(2.0, 0, 0),
        timing_dependence=sim.SpikePairRule(
            tau_plus=20.0, tau_minus=20.0, A_plus=0.01, A_minus=0.01),
        weight_dependence=sim.AdditiveWeightDependence(w_min, w_max),
        f_rew=1000, initial_weight=w_init_1, initial_delay=delay_1,
        s_max=1, seed=0, weight=0.0, delay=1.0),
    download_synapses=True)
sim.run(10)
sim.end()

## Exercises
The following exercises are designed to enhance your understanding of PyNN.  You have the option of following the step-by-step instructions in the linked notebooks, or you can try to follow the instructions below each task (this will be harder).

 - [Task 1: A simple neural network (Easy)](task1.ipynb)
    - Create a network with a timestep of 1.0ms consisting of 2 SpikeSourceArray neurons, spiking at 0.0ms and 1.0ms respectively, connected to 2 LIF neurons with a one-to-one connector, using a weight of 5.0nA and a delay of 2.0ms.  Record the spikes and run for 10ms, plotting the spikes after the simulation has completed.
    - See the notebook for further extensions.
 - [Task 2: Synfire Chain (Moderate)](task2.ipynb)
    - Create a "synfire chain" network from a single Population of 100 neurons using a FromListConnector to connect each neuron to the next neuron in the Population, additionally looping back around from the last to the first.  Use a SpikeSourceArray to stimulate the first neuron in the Population at the start.  Record and plot the spikes after running for 2 seconds.
    - See the notebook for further extensions.
 - [Task 3: Balanced Random Cortex-like Network (Hard)](task3.ipynb)
    - Create a "balanced random network" with a timestep of 0.1ms consisting of one "excitatory" and one "inhibitory" Population of LIF neurons with a size in the ratio of 4:1, and stimulate each in a one-to-one fashion using a SpikeSourcePoisson with rate 1000Hz.  Make the excitatory Population send excitatory spikes to itself and the inhibitory Population, and the inhibitory Population send inhibitory spikes to itself and the excitatory population.  Initialize the voltages using a random uniform distribution between -65.0 and -55.0.  Record and plot the spikes from the excitatory Population.
    - See the notebook for further extensions.
 - [Task 4: STDP Network (Moderate)](task4.ipynb)
     - Create a network of two single-neuron LIF Populations connected by an STDP connection, which has a tau_plus larger than tau_minus.  Stimulate each of the Populations repeatedly with gaps between stimulations, the first just before the second.  Plot the spikes of each.
     - See the notebook for further extensions.
 - [Task 5: STDP Curve (Hard)](task5.ipynb)
     - Create an STDP curve graph using a SpikeSourceArray stimulating a LIF population, with varying spike times for the SpikeSourceArray.
     - See the notebook for further extensions.
